In [5]:
import requests
from bs4 import BeautifulSoup
from tqdm.asyncio import tqdm
import asyncio
import json
import requests

In [6]:
url = "https://www.apu.edu.my/sitemap.xml"
response = requests.get(url)
soup = BeautifulSoup(response.content, "xml")

locs = soup.find_all('loc')
urls = [loc.get_text() for loc in locs]

print(len(urls))

418


In [7]:
ignore = ["http://www.apu.edu.my/china", "http://www.apu.edu.my/our-courses-chi"]
dataset = []

async def crawl(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "lxml")
    title = soup.find('h1', {'id': 'page-title'})
    if url in ignore or title == None or title.text.strip() == "Access denied":
        return
    data = {
        'url': url,
        'title': title.text.strip(),
        'body': soup.find('div', {'class': 'content-middle'}).get_text(separator="\n"),
    }
    dataset.append(data)

async def main():
    flist = [crawl(url) for url in urls]
    for f in tqdm(flist):
        await f

In [8]:
await main()

with open('apu.edu.jsonl', 'w') as file:
    for entry in dataset:
        json.dump(entry, file)
        file.write('\n')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 418/418 [01:47<00:00,  3.89it/s]
